In [1]:
# Pygame installation
!pip install pygame

In [2]:
import pygame as pg
from pygame import image as img

pg.init()

pygame 2.1.0 (SDL 2.0.16, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


(5, 0)

In [3]:
# Function definitions for use
def load_image(file):
    """loads an image, prepares it for play"""
    try:
        surface = pg.image.load(file)
    except pg.error:
        raise SystemExit('Could not load image "%s" %s' % (file, pg.get_error()))
    return surface.convert_alpha() # convert_alpha allows for transparency from .pngs

In [78]:
pg.init()
win = pg.display.set_mode((0,0))
pg.display.set_caption("Custom Environment - Space Path")

class Robot(pg.sprite.Sprite):
    """Visual for AI space and movement direction"""
    images = [load_image("assets/sprites/robo_R.png"), 
              load_image("assets/sprites/robo_L.png"), 
              load_image("assets/sprites/robo_U.png"),
              load_image("assets/sprites/robo_D.png")]

    def __init__(self, pos):
        pg.sprite.Sprite.__init__(self, self.containers)
        self.image = self.images[0]
        self.rect = self.image.get_rect()
        self.rect.x = pos[0] - (self.rect.width / 2.0)
        self.rect.y = pos[1] - (self.rect.height / 2.0)

    def update(self, action: int):
        if action >= 0:
            self.image = self.images[action]
    
    def setPosition(self, newPos):
        self.rect = self.image.get_rect()
        self.rect.x = newPos[0] - (self.rect.width / 2.0)
        self.rect.y = newPos[1] - (self.rect.height / 2.0)

class Space(pg.sprite.Sprite):
    """Visual for space type. Normal, positive or negative"""
    images = [load_image("assets/sprites/normal.png"), 
              load_image("assets/sprites/plus.png"), 
              load_image("assets/sprites/minus.png"),
              load_image("assets/sprites/start.png"), 
              load_image("assets/sprites/goal.png")]

    def __init__(self, pos, jumpDistance):
        pg.sprite.Sprite.__init__(self, self.containers)
        
        if(jumpDistance == 0):  # neutral movement - does not push Robot
            self.image = self.images[0]
        elif(jumpDistance > 0): # positive movement - pushes robot forward
            self.image = self.images[1]
        else:                    # negative movement - pushes robot backward
            self.image = self.images[2]

        self.rect = self.image.get_rect()
        self.rect.x = pos[0] - (self.rect.width / 2.0)
        self.rect.y = pos[1] - (self.rect.height / 2.0)
        
    def setType(self, spaceType):
            self.image = self.images[spaceType]

# Initialize Game Groups
all = pg.sprite.RenderUpdates()
    
Space.containers = all
Robot.containers = all

# Board does not require rendering
        
class Board():
    """Board that sets up and displays all spaces"""
    spaces = [[], [], []] # top, middle and bottom lanes
    playerPos = [0,1]
    goal = []
    maxRows = 0
    
    def __init__(self, maxRows, pos):
        self.maxRows = maxRows
        
        if self.maxRows < 1:
            self.maxRows = 1 # cap minimum value to max rows in case of emergency
            
        self.goal = [1, self.maxRows - 1]
        
        # Thanks to Jack Malone for help with this loop
        for index, value in enumerate(self.spaces):
                    for y in range(self.maxRows):
                            value.append(Space(((64 * y) + pos[0], (64 * index) + pos[1]), 0))
                        
        self.spaces[self.playerPos[0]][self.playerPos[1]].setType(3) # 3 = start type
        self.spaces[self.goal[0]][self.goal[1]].setType(4) # 4 = goal type
            
    def update(self, robot: Robot, action: int):
        if action >= 0:
            if action == 0: # right
                  if self.playerPos[0] < self.maxRows - 1:
                        self.playerPos[0] += 1
            elif action == 1: # left
                 if self.playerPos[0] > 0:
                        self.playerPos[0] -= 1
            elif action == 2: # up
                if self.playerPos[1] > 0:
                        self.playerPos[1] -= 1
            elif action == 3: # down
                if self.playerPos[1] < 2:
                        self.playerPos[1] += 1
            # now that the robot has moved, update it's position
            robot.setPosition(((self.playerPos[0] * 64) + 96, (self.playerPos[1] * 64) + 96))
        

pg.display.quit()

In [79]:
# Game Setup
pg.init()
win = pg.display.set_mode((1600,800))
pg.display.set_caption("Custom Env")

bg = load_image('assets/sprites/background.png')
board = Board(6, (100,100))
robo = Robot((board.spaces[0][1].rect.x  + 96, board.spaces[0][1].rect.y  + 96))

clock = pg.time.Clock()

run = True

def draw():
    win.blit(bg, (0,0))
    win.blit(robo.image, (robo.rect.x, robo.rect.y))
    
    for r in board.spaces:
            for c in r: # position all spaces to the correct top left position
                win.blit(c.image, (c.rect.x, c.rect.y))
    
    
    pg.display.update()


while run:
    clock.tick(30)
    
    for event in pg.event.get():
        if event.type == pg.QUIT:
            run = False
        elif event.type == pg.KEYUP :
            if keys[pg.K_RIGHT]:
                action = 0   
            elif keys[pg.K_LEFT]:
                action = 1   
            elif keys[pg.K_UP]:
                action = 2 
            elif keys[pg.K_DOWN]:
                action = 3
            
            
    keys = pg.key.get_pressed()

    
    robo.update(action)
    board.update(robo, action)
    
    action = -1;
    
    draw()
        
    
pg.quit()

In [69]:
pg.quit()